In [1]:
import torch
import os
import sys
import pandas as pd
import numpy as np
import random
import os.path
from pathlib import Path
from collections import Counter

from datasets.cifar10 import Cifar10 
from datasets.from_files import FromFiles 
from models.cifar10_resnet101_architecture import Cifar10Resnet101Architecture 
from models.cifar10_resnet101_gap import Cifar10Resnet101Gap
from models.cifar10_resnet101_crow import Cifar10Resnet101CroW
from models.cifar10_simple_architecture import Cifar10SimpleArchitecture 
from models.cifar10_simple_gap import Cifar10SimpleGap
from ood.calculate_metrics import CalculateMetrics

from ood.max_softmax import MaxSoftmax
from ood.max_logits import MaxLogits
from ood.lofdiv import LOFDIV
from ood.lof import LOF
from ood.mahalanobis import Mahalanobis
from ood.osnn import Osnn
from ood.our_distance import OurDistance

np.random.seed(0)
torch.manual_seed(0)
random.seed(0)

os.environ["CUDA_VISIBLE_DEVICES"]=str(1); 
device = torch.device("cuda:0")

batch_size = 64

cifar10_orginal = Cifar10()
cifar10 = FromFiles("hard_cifar10", "./results/cifar10/known/")
cifar100 = FromFiles("hard_cifar100", "./results/cifar10/unknown/cifar100/")
svhn = FromFiles("hard_svhn", "./results/cifar10/unknown/svhn/")
mnist = FromFiles("hard_mnist", "./results/cifar10/unknown/mnist/")
noise = FromFiles("hard_noise", "./results/cifar10/unknown/noise/")

In [2]:
def get_df(loader):
    all_out = []
    last_id = 0
    for i_batch, (data, targets) in enumerate(loader):
        sys.stdout.write("{}\r".format(i_batch))
        sys.stdout.flush()
                
        data = data.to(device)

        features = extraction.obtain_features(data)
        outputs = model(data)
        for i in range(len(data)):
            out = {}
            out["id"] = last_id + i
            out["original_label"] = targets[i].item()
            out["features"] = np.array(features[i].detach().cpu())
            out["classifier"] = np.array(outputs[i].detach().cpu())
            all_out.append(out)
        last_id += len(data)
                        
    df = pd.DataFrame(all_out)
    df["predicted_label"] = df["classifier"].apply(lambda x: np.argmax(x))
    print("            Done: df", len(df))
    return df

def test_ood(ood):
    calculate_metrics = CalculateMetrics()
   
    ood.clear()
    ood.fit(train_df)
    known_results = ood.test(known_df)
    unknown_results = ood.test(unknown_df)

    bin_acc, wrong_known, wrong_unknown = calculate_metrics.run(ood, known_results, unknown_results)

    print(model.name, extraction.name, ood.name, "---", bin_acc, len(wrong_known), len(wrong_unknown))
    

def itr_merge(*itrs):
    for itr in itrs:
        for v in itr:
            yield v

In [3]:
known_set = cifar10

train_loader = cifar10_orginal.get_train_loader(batch_size)
known_loader = cifar10.get(batch_size)

cifar100_loader = cifar100.get(batch_size)
svhn_loader = svhn.get(batch_size)
mnist_loader = mnist.get(batch_size)
noise_loader = noise.get(batch_size)

Files already downloaded and verified


In [4]:
cifar10_resnet101 = Cifar10Resnet101Architecture()
cifar10_resnet101.load_state_dict(torch.load("./models/cifar10_resnet101.pt"), strict=False)
cifar10_resnet101 = cifar10_resnet101.to(device)
cifar10_resnet101.eval()
cifar10_resnet101_gap = Cifar10Resnet101Gap(cifar10_resnet101)
cifar10_resnet101_crow = Cifar10Resnet101CroW(cifar10_resnet101)

cifar10_simple = Cifar10SimpleArchitecture()
cifar10_simple.load_state_dict(torch.load("./models/cifar10_simple.pt"), strict=False)
cifar10_simple = cifar10_simple.to(device)
cifar10_simple.eval()
cifar10_simple_gap = Cifar10SimpleGap(cifar10_simple)

In [6]:
model = cifar10_resnet101
extraction = cifar10_resnet101_gap

unknown_loader = itr_merge(cifar100_loader, svhn_loader, mnist_loader, noise_loader)
train_df = get_df(train_loader)
known_df = get_df(known_loader)
unknown_df = get_df(unknown_loader)

test_ood(MaxSoftmax())
test_ood(MaxLogits())
test_ood(Mahalanobis(is_robust=False))
test_ood(LOF(LOF.EUCLIDEAN))
test_ood(LOF(LOF.COSINE))
test_ood(LOFDIV(LOFDIV.EUCLIDEAN))
test_ood(LOFDIV(LOFDIV.COSINE))
test_ood(Osnn(Osnn.EUCLIDEAN))
test_ood(Osnn(Osnn.COSINE))
test_ood(OurDistance(OurDistance.EUCLIDEAN))
test_ood(OurDistance(OurDistance.COSINE))

Cifar10Resnet101 gap MaxSoftmax --- 0.5 0 1000
Cifar10Resnet101 gap MaxLogits --- 0.5 0 1000
Cifar10Resnet101 gap mahalanobis --- 0.5 1000 0
Cifar10Resnet101 gap lof_euclidean --- 0.5 1000 0
Cifar10Resnet101 gap lof_cosine --- 0.5 1000 0
Cifar10Resnet101 gap lofdiv_euclidean --- 0.5 1000 0
Cifar10Resnet101 gap lofdiv_cosine --- 0.5 1000 0
Cifar10Resnet101 gap OSNN_euclidean --- 0.5 0 1000
Cifar10Resnet101 gap OSNN_cosine --- 0.5 0 1000


KeyboardInterrupt: 

In [9]:
model = cifar10_resnet101
extraction = cifar10_resnet101_crow

unknown_loader = itr_merge(cifar100_loader, svhn_loader, mnist_loader, noise_loader)
train_df = get_df(train_loader)
known_df = get_df(known_loader)
unknown_df = get_df(unknown_loader)

###

test_ood(MaxSoftmax())
test_ood(MaxLogits())
test_ood(Mahalanobis(is_robust=False))
test_ood(LOF(LOF.EUCLIDEAN))
test_ood(LOF(LOF.COSINE))
test_ood(LOFDIV(LOFDIV.EUCLIDEAN))
test_ood(LOFDIV(LOFDIV.COSINE))
test_ood(Osnn(Osnn.EUCLIDEAN))
test_ood(Osnn(Osnn.COSINE))
test_ood(OurDistance(OurDistance.EUCLIDEAN))
test_ood(OurDistance(OurDistance.COSINE))

            Done: df 1000
Cifar10Resnet101 CroW MaxSoftmax --- 0.5 0 1000
Cifar10Resnet101 CroW MaxLogits --- 0.5 0 1000
Cifar10Resnet101 CroW mahalanobis --- 0.5 1000 0
Cifar10Resnet101 CroW lof_euclidean --- 0.5 1000 0
Cifar10Resnet101 CroW lof_cosine --- 0.5 1000 0
Cifar10Resnet101 CroW lofdiv_euclidean --- 0.5 1000 0
Cifar10Resnet101 CroW lofdiv_cosine --- 0.5 1000 0
Cifar10Resnet101 CroW OSNN_euclidean --- 0.5 0 1000
Cifar10Resnet101 CroW OSNN_cosine --- 0.5 0 1000


KeyboardInterrupt: 

In [11]:
model = cifar10_simple
extraction = cifar10_simple_gap

unknown_loader = itr_merge(cifar100_loader, svhn_loader, mnist_loader, noise_loader)
train_df = get_df(train_loader)
known_df = get_df(known_loader)
unknown_df = get_df(unknown_loader)

###

test_ood(MaxSoftmax())
test_ood(MaxLogits())
test_ood(Mahalanobis(is_robust=False))
test_ood(LOF(LOF.EUCLIDEAN))
test_ood(LOF(LOF.COSINE))
test_ood(LOFDIV(LOFDIV.EUCLIDEAN))
test_ood(LOFDIV(LOFDIV.COSINE))
test_ood(Osnn(Osnn.EUCLIDEAN))
test_ood(Osnn(Osnn.COSINE))
test_ood(OurDistance(OurDistance.EUCLIDEAN))
test_ood(OurDistance(OurDistance.COSINE))

            Done: df 50000
            Done: df 1000
            Done: df 1000
Cifar10Simple gap MaxSoftmax --- 0.5 0 1000
Cifar10Simple gap MaxLogits --- 0.5005 998 1
Cifar10Simple gap mahalanobis --- 0.584 87 745
Cifar10Simple gap lof_euclidean --- 0.6095 67 714
Cifar10Simple gap lof_cosine --- 0.594 435 377
Cifar10Simple gap lofdiv_euclidean --- 0.618 60 704
Cifar10Simple gap lofdiv_cosine --- 0.562 506 370
Cifar10Simple gap OSNN_euclidean --- 0.522 403 553
Cifar10Simple gap OSNN_cosine --- 0.5065 388 599
Cifar10Simple gap our_distance_euclidean --- 0.589 212 610
Cifar10Simple gap our_distance_cosine --- 0.5 1 999
